# Hackathon Demonstration
Beyond the two previous examples, which closely followed the Langchain [custom agent](https://python.langchain.com/docs/how_to/custom_tools/) tutorial, the goal for the final presentation was to highlight the use of `LangSim` in production. For this purpose, the functionality was transferred from the Jupyter notebook to a Python module. As a first step the `LangSim` module is loaded using the `%reload_ext` jupyter magic command:

In [1]:
%reload_ext langsim

Then the [Open AI](http://openai.com) API key is set to the `OPENAI_API_KEY` environment variable so it can be used by `LangSim` internally:

In [2]:
import os, getpass
os.environ['OPENAI_API_KEY'] = getpass.getpass("Enter your API key")

Enter your API key ········


## Dialog
From this point on the user can chat with the `LangSim` interface using the `chat` jupyter magic. As a first challenge the user communicates their interest to find a material with a bulk modulus in the range of 200 GPa; following with a clarification of this statement by asking the LLM to calculate the bulk modulus for the noble metals. In contrast to the previous example, the LLM is tasked with repeating the same calculation multiple times and then identify the chemical element, which most closely matches the targeted bulk modulus of 200GPa. None of the required logic is implemented explicitly, rather the LLM only has access to a `get_equilibrium_lattice()` function to calculate the equilibrium crystal structure and a `get_bulk_modulus()` function, which calculates the bulk modulus for a given equilibrium crystal structure.

In [3]:
%%chat
I am looking for a material with a bulk modulus in the range of 200 GPa.
Can you compute the bulk modulus for the noble metals for me?



> Entering new AgentExecutor chain...
Sure, I can help with that. The noble metals include gold (Au), silver (Ag), and platinum (Pt). Which model would you like to use for the calculations: EMT or MACE?

> Finished chain.


Sure, I can help with that. The noble metals include gold (Au), silver (Ag), and platinum (Pt). Which model would you like to use for the calculations: EMT or MACE?

The LLM is instructed by the system prompt to clarify which simulation method it should use to calculate a given material property. At the time of the demonstration, two simulation methods were implemented, [EMT](https://wiki.fysik.dtu.dk/ase/ase/calculators/emt.html) and [MACE](https://mace-docs.readthedocs.io). As the user in this demonstration is not aware of the technical details of these simulation methods, he asks for clarification. This highlights how LLMs can use the context of the previous conversation to answer a given question:

In [4]:
%%chat 
What are those methods?



> Entering new AgentExecutor chain...
Certainly! Here is a brief overview of the two models:

1. **Effective Medium Theory (EMT)**:
   - **Description**: EMT is a computationally efficient, analytical model that describes the macroscopic properties of composite materials.
   - **Advantages**: It is relatively fast and can provide good approximations for certain types of materials.
   - **Limitations**: It may not capture all the complexities of atomic interactions, especially for materials with complex bonding.

2. **MACE (Machine Learning Force Field)**:
   - **Description**: MACE is a machine learning force field for predicting many-body atomic interactions that covers the periodic table.
   - **Advantages**: It leverages machine learning to provide highly accurate predictions of atomic interactions and material properties.
   - **Limitations**: It can be computationally more intensive compared to EMT.

Please choose one of these models for the calculations.

> Finished chain.


Certainly! Here is a brief overview of the two models:

1. **Effective Medium Theory (EMT)**:
   - **Description**: EMT is a computationally efficient, analytical model that describes the macroscopic properties of composite materials.
   - **Advantages**: It is relatively fast and can provide good approximations for certain types of materials.
   - **Limitations**: It may not capture all the complexities of atomic interactions, especially for materials with complex bonding.

2. **MACE (Machine Learning Force Field)**:
   - **Description**: MACE is a machine learning force field for predicting many-body atomic interactions that covers the periodic table.
   - **Advantages**: It leverages machine learning to provide highly accurate predictions of atomic interactions and material properties.
   - **Limitations**: It can be computationally more intensive compared to EMT.

Please choose one of these models for the calculations.

Based on the explanation of the LLM, the user chooses the [EMT](https://wiki.fysik.dtu.dk/ase/ase/calculators/emt.html) method, primarily because it is computationally more efficient to calculate a large number of calculations in a short amount of time. The LLM then executes two calculations for each of the three noble metals silver (Ag), gold (Au) and platinum (Pt). These calculations are the calculation of the equilibrium structure `get_equilibrium_lattice()` which in addition to the chemical symbol now also takes the name of the chosen calculator as an input and the calculation of the bulk modulus `get_bulk_modulus()` which takes the optimized structure as `AtomsDict()` object as an input in addition to the chosen calculator as string. 

In [5]:
%%chat
Let's take EMT then



> Entering new AgentExecutor chain...

Invoking: `get_atom_dict_bulk_structure` with `{'chemical_symbol': 'Au'}`


numbers=[79] positions=[[0.0, 0.0, 0.0]] cell=[[0.0, 2.04, 2.04], [2.04, 0.0, 2.04], [2.04, 2.04, 0.0]] pbc=[True, True, True]
Invoking: `get_atom_dict_bulk_structure` with `{'chemical_symbol': 'Ag'}`


numbers=[47] positions=[[0.0, 0.0, 0.0]] cell=[[0.0, 2.045, 2.045], [2.045, 0.0, 2.045], [2.045, 2.045, 0.0]] pbc=[True, True, True]
Invoking: `get_atom_dict_bulk_structure` with `{'chemical_symbol': 'Pt'}`


numbers=[78] positions=[[0.0, 0.0, 0.0]] cell=[[0.0, 1.96, 1.96], [1.96, 0.0, 1.96], [1.96, 1.96, 0.0]] pbc=[True, True, True]
Invoking: `get_atom_dict_equilibrated_structure` with `{'atom_dict': {'numbers': [79], 'positions': [[0.0, 0.0, 0.0]], 'cell': [[0.0, 2.04, 2.04], [2.04, 0.0, 2.04], [2.04, 2.04, 0.0]], 'pbc': [True, True, True]}, 'calculator_str': 'emt'}`


       Step     Time          Energy          fmax
LBFGS:    0 19:41:36        0.002606        0.30885

Here are the calculated bulk moduli for the noble metals using the EMT model:

- **Gold (Au)**: 173.84 GPa
- **Silver (Ag)**: 100.16 GPa
- **Platinum (Pt)**: 278.30 GPa

Among these, Platinum (Pt) has a bulk modulus within the range of 200 GPa.

The results are summarized in a short list. This demonstration highlights that `LangSim` can be used without prior technical experience in calculating material properties. From students to teachers and professors, it is a handy tool for everybody who wants to have quick access to material properties computed with one of the standard methods like [EMT](https://wiki.fysik.dtu.dk/ase/ase/calculators/emt.html). Obviously, the [MACE](https://mace-docs.readthedocs.io) method is more accurate than the [EMT](https://wiki.fysik.dtu.dk/ase/ase/calculators/emt.html) method, but also computationally more expensive. 

Still, to further highlight the real-life application of a tool like `LangSim` the next step after the completion of these calculations is the comparison to an experimental reference. For this purpose, the `get_experimental_elastic_property_wikipedia()` function is implemented which downloads the experimental values for the elastic properties from the wikipedia site for [elastic properties](https://en.wikipedia.org/wiki/Elastic_properties_of_the_elements_(data_page)).

In [6]:
%%chat
How does this compare to experiment?



> Entering new AgentExecutor chain...

Invoking: `get_experimental_elastic_property_wikipedia` with `{'chemical_symbol': 'Au'}`


{'youngs_modulus': 78.0, 'poissons_ratio': 0.44, 'bulk_modulus': 180.0, 'shear_modulus': 27.0}
Invoking: `get_experimental_elastic_property_wikipedia` with `{'chemical_symbol': 'Ag'}`


{'youngs_modulus': 83.0, 'poissons_ratio': 0.37, 'bulk_modulus': 100.0, 'shear_modulus': 30.0}
Invoking: `get_experimental_elastic_property_wikipedia` with `{'chemical_symbol': 'Pt'}`


{'youngs_modulus': 168.0, 'poissons_ratio': 0.38, 'bulk_modulus': 230.0, 'shear_modulus': 61.0}Here are the experimental bulk moduli for the noble metals compared to the calculated values using the EMT model:

- **Gold (Au)**:
  - **Calculated (EMT)**: 173.84 GPa
  - **Experimental**: 180.0 GPa

- **Silver (Ag)**:
  - **Calculated (EMT)**: 100.16 GPa
  - **Experimental**: 100.0 GPa

- **Platinum (Pt)**:
  - **Calculated (EMT)**: 278.30 GPa
  - **Experimental**: 230.0 GPa

The calculated valu

Here are the experimental bulk moduli for the noble metals compared to the calculated values using the EMT model:

- **Gold (Au)**:
  - **Calculated (EMT)**: 173.84 GPa
  - **Experimental**: 180.0 GPa

- **Silver (Ag)**:
  - **Calculated (EMT)**: 100.16 GPa
  - **Experimental**: 100.0 GPa

- **Platinum (Pt)**:
  - **Calculated (EMT)**: 278.30 GPa
  - **Experimental**: 230.0 GPa

The calculated values are quite close to the experimental values, with the exception of Platinum (Pt), which shows a larger discrepancy.

Finally, as the last step the user tasks the LLM to compare the theoretical predictions and the experimental results to give an indication of the trustworthiness of these calculations. Given the optimization for computational efficiency, the result for Platinum with an error of over 20% is insufficient for any real life application. Still, this can be addressed by switching to the [MACE](https://mace-docs.readthedocs.io) model. 

In [7]:
%%chat
Can you show me the deviation in percent?



> Entering new AgentExecutor chain...
Certainly! Let's calculate the percentage deviation between the calculated and experimental bulk moduli for the noble metals.

The formula for percentage deviation is:
\[ \text{Percentage Deviation} = \left( \frac{\text{Calculated Value} - \text{Experimental Value}}{\text{Experimental Value}} \right) \times 100 \]

Here are the calculations:

1. **Gold (Au)**:
   - Calculated (EMT): 173.84 GPa
   - Experimental: 180.0 GPa
   - Percentage Deviation: \[ \left( \frac{173.84 - 180.0}{180.0} \right) \times 100 \approx -3.42\% \]

2. **Silver (Ag)**:
   - Calculated (EMT): 100.16 GPa
   - Experimental: 100.0 GPa
   - Percentage Deviation: \[ \left( \frac{100.16 - 100.0}{100.0} \right) \times 100 \approx 0.16\% \]

3. **Platinum (Pt)**:
   - Calculated (EMT): 278.30 GPa
   - Experimental: 230.0 GPa
   - Percentage Deviation: \[ \left( \frac{278.30 - 230.0}{230.0} \right) \times 100 \approx 21.87\% \]

So, the percentage deviations are:
- **Gold (Au)**: 

Certainly! Let's calculate the percentage deviation between the calculated and experimental bulk moduli for the noble metals.

The formula for percentage deviation is:
\[ \text{Percentage Deviation} = \left( \frac{\text{Calculated Value} - \text{Experimental Value}}{\text{Experimental Value}} \right) \times 100 \]

Here are the calculations:

1. **Gold (Au)**:
   - Calculated (EMT): 173.84 GPa
   - Experimental: 180.0 GPa
   - Percentage Deviation: \[ \left( \frac{173.84 - 180.0}{180.0} \right) \times 100 \approx -3.42\% \]

2. **Silver (Ag)**:
   - Calculated (EMT): 100.16 GPa
   - Experimental: 100.0 GPa
   - Percentage Deviation: \[ \left( \frac{100.16 - 100.0}{100.0} \right) \times 100 \approx 0.16\% \]

3. **Platinum (Pt)**:
   - Calculated (EMT): 278.30 GPa
   - Experimental: 230.0 GPa
   - Percentage Deviation: \[ \left( \frac{278.30 - 230.0}{230.0} \right) \times 100 \approx 21.87\% \]

So, the percentage deviations are:
- **Gold (Au)**: -3.42%
- **Silver (Ag)**: 0.16%
- **Platinum (Pt)**: 21.87%

These deviations indicate how close the calculated values are to the experimental values.

## Summary
The jury was convinced that this demonstration of the capabilities of `LangSim` highlights a future direction in the application of LLMs for materials and chemistry. So the `LangSim` team won the [first prize](https://medium.com/@blaiszik/llms-to-accelerate-discovery-in-materials-science-and-chemistry-refections-on-a-hackathon-b8364ca32242) sponsored by [RadicalAI](https://www.radical-ai.com). 

In comparison to the previous examples, the highlight of this presentation, besides hiding the technical complexity by developing a python module, was the capability of the LLM to repeat the workflow consisting of two calculations, the `get_equilibrium_lattice()` function and the `get_bulk_modulus()` function, over three elements without the need to define any additional functionality. 